### `dist-data` Data Viewer
#### Rev. 8/23/2022.

In [ ]:
import au
import h5py
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

# data directory
run_path = '/media/ayounis/DATA1/qray/1d2e/2021/run3'

### Load momentum / position distribution data

In [ ]:
# distribution
mode = 'p'

# Momentum distribution
if (mode == 'p'):
    #f = h5py.File(run_path+'/dist_data/pdist.h5','r')
    f = h5py.File(run_path+'/out/dist_data/pdist.h5','r')
    
    p1 = np.array(f['p1']); dp1 = p1[1]-p1[0];
    p2 = np.array(f['p2']); dp2 = p2[1]-p2[0];
    extent_p = [p1[0],p1[-1],p2[0],p2[-1]]
    
    W0 = np.array(f['W0'])
    Wp = np.abs(np.array(f['Re(W)']) + 1j*np.array(f['Im(W)']))**2
    # normalize
    W0 = (W0-np.min(W0))/(np.max(W0)-np.min(W0))
    Wp = (Wp-np.min(Wp))/(np.max(Wp)-np.min(Wp))
    # log10
    logWp = np.ma.log10(Wp).filled(-50)
    logW0 = np.ma.log10(W0).filled(-50)
    
    nde = np.array(f['nde'])
    print('\nMomentum distribution')
    print(str(np.round(nde*1e-6,3)) + ' million virtual particles')
    print('bins: ' + str(np.shape(p1)[0]))

# Position distribution
if (mode == 'x'):
    f = h5py.File(run_path+'/dist_data/xdist.h5','r')
    
    x1 = np.array(f['x1']); dx1 = x1[1]-x1[0];
    x2 = np.array(f['x2']); dx2 = x2[1]-x2[0];
    extent_x = [x1[0],x1[-1],x2[0],x2[-1]]
    
    W0 = np.array(f['W0'])
    Wp = np.abs(np.array(f['Re(W)']) + 1j*np.array(f['Im(W)']))**2
    # normalize
    W0 = (W0-np.min(W0))/(np.max(W0)-np.min(W0))
    Wp = (Wp-np.min(Wp))/(np.max(Wp)-np.min(Wp))
    # log10
    logWp = np.ma.log10(Wp).filled(-50)
    logW0 = np.ma.log10(W0).filled(-50)
    
    nde = np.array(f['nde'])
    print('\nPosition distribution')
    print(str(nde*1e-6) + ' million virtual particles')
    print('bins: ' + str(np.shape(x1)[0]))

### Plotter settings

In [ ]:
font = {'size':22,'weight':'normal'}

matplotlib.rc('font',**font)
dpi = 75 # 300 for production

# toggle PNG/EPS export
save_PNG = False
save_EPS = False

### Plot momentum distribution

In [ ]:
# viewer settings
view_type = 'log'
view_cmap = 'turbo'
# axes line-width
axlw = 1
# color-axis limits, cbar shrink, pad
if view_type == 'log': clim = [np.min(logWp)+10,np.max(logWp)+5]
if view_type == 'log': clim = [-8,0]
if view_type == 'lin': clim = [0,1e-2]
shrink = 0.5; pad = 0.025;

plt.figure(figsize=(8,8), dpi=dpi, facecolor='w')

if (view_type == 'log'):
    plt.imshow(logWp, extent=extent_p, origin='lower', cmap=view_cmap)
if (view_type == 'lin'):
    plt.imshow(Wp, extent=extent_p, origin='lower', cmap=view_cmap)

plt.plot([0,0], [p2[0],p2[-1]], '-', c='k', lw=axlw)
plt.plot([p1[0],p1[-1]], [0,0], '-', c='k', lw=axlw)

cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=[]); cbar.ax.tick_params(labelsize=font['size']); plt.clim(clim);
if (view_type == 'log'): cbar.set_label('$\\log W(\\vec{p})$', labelpad=10, fontsize=font['size']-4);
if (view_type == 'lin'): cbar.set_label('$W(\\vec{p})$', labelpad=10, fontsize=font['size']-4);

plt.xlim([p1[0]+0.015*p1[0], p1[-1]+0.015*p1[-1]]), plt.ylim([p2[0]+0.015*p2[0], p2[-1]+0.015*p2[-1]])

plt.xlabel('P${}_1$ (a.u.)', labelpad=8)
plt.ylabel('P${}_2$ (a.u.)', labelpad=0)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

plt.tight_layout()
if (save_PNG): plt.savefig('pdist_'+str(view_type)+'.png', dpi=dpi)
if (save_EPS): plt.savefig('pdist_'+str(view_type)+'.eps', format='eps')
plt.show()

In [ ]:
l = np.shape(Wp)[0]

'''Integrate W(p) in quadrants II and IV'''
Qn = dp1*dp2*np.trapz(np.trapz(Wp[int(l/2):, 0:int(l/2)])) \
   + dp1*dp2*np.trapz(np.trapz(Wp[0:int(l/2), int(l/2):]))
'''Integrate W(p) in quadrants I and III'''
Qp = dp1*dp2*np.trapz(np.trapz(Wp[int(l/2):, int(l/2):])) \
   + dp1*dp2*np.trapz(np.trapz(Wp[0:int(l/2), 0:int(l/2)]))

print('Probability(p1*p2 < 0) = ' + str(np.round(1e+2*Qn/(Qn+Qp),3)) + '%')

### Plot position distribution

In [ ]:
# viewer settings
view_type = 'log'
view_cmap = 'turbo'
# axes line-width
axlw = 1
# color-axis limits, cbar shrink, pad
if view_type == 'log': clim = [np.min(logWp),np.max(logWp)+10]
if view_type == 'lin': clim = [0,5e-3]
shrink = 0.5; pad = 0.025;

plt.figure(figsize=(8,8), dpi=dpi, facecolor='w')

if (view_type == 'log'):
    plt.imshow(logWp, extent=extent_x, origin='lower', cmap=view_cmap)
if (view_type == 'lin'):
    plt.imshow(Wp, extent=extent_x, origin='lower', cmap=view_cmap)

plt.plot([0,0], [x2[0],x2[-1]], '-', c='w', lw=axlw)
plt.plot([x1[0],x1[-1]], [0,0], '-', c='w', lw=axlw)

cbar = plt.colorbar(shrink=shrink, pad=pad, ticks=[]); cbar.ax.tick_params(labelsize=font['size']); plt.clim(clim);
if (view_type == 'log'): cbar.set_label('$\\log W(\\vec{x})$', labelpad=10, fontsize=font['size']-4);
if (view_type == 'lin'): cbar.set_label('$W(\\vec{x})$', labelpad=10, fontsize=font['size']-4);

plt.xlim([x1[0]+0.015*x1[0], x1[-1]+0.015*x1[-1]]), plt.ylim([x2[0]+0.015*x2[0], x2[-1]+0.015*x2[-1]])

plt.xlabel('X${}_1$ ($a_0$)', labelpad=12)
plt.ylabel('X${}_2$ ($a_0$)', labelpad=-5)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
ax.xaxis.set_major_locator(MultipleLocator(1e+3))
ax.xaxis.set_minor_locator(MultipleLocator(2e+2))
ax.yaxis.set_major_locator(MultipleLocator(1e+3))
ax.yaxis.set_minor_locator(MultipleLocator(2e+2))

plt.tight_layout()
if (save_PNG): plt.savefig('xdist_'+str(view_type)+'.png', dpi=dpi)
if (save_EPS): plt.savefig('xdist_'+str(view_type)+'.eps', format='eps')
plt.show()

### Energy lineout diagnostic

In [ ]:
# transverse averaging samples
tas = 15

yE = np.zeros(np.shape(Wp)[0])
for i in range(tas):
    yE += Wp[i+int(np.shape(Wp)[0]/2)-int(tas/2),:]
yE = (yE-np.min(yE))/(np.max(yE)-np.min(yE))

# laser photon energy
omg = 0.0584

xE = p1**2/2.0/omg

### Plot energy lineout

In [ ]:
plt.figure(figsize=(14,8), dpi=dpi, facecolor='w')
plt.plot(xE[int(len(p1)/2)+1:], yE[int(len(p1)/2):], 'k', lw=1.5)

plt.xlabel('Energy ($\omega$)', labelpad=15)
plt.ylabel('Yield (arb. u.)', labelpad=20)

plt.xticks(range(13))

plt.xlim([0,12]);
plt.ylim(bottom=0);

plt.tight_layout()
if (save_PNG): plt.savefig('eer.png', dpi=dpi)
if (save_EPS): plt.savefig('eer.eps', format='eps')
plt.show()